## Using Pandas Categoricals

possible to use categorical dtypes to carry "set metadata" around, instead of external set logic or dataframes. Then, indices can be passed around to consistently slice different dataframes in a bigger Xarray or Anndata object....

In [1]:
import pandas as pd
import numpy as np

In [97]:
letters=pd.CategoricalDtype(list('abcdefghijklmnop'))
numbers=pd.CategoricalDtype(range(10))

In [110]:
multi=pd.MultiIndex.from_arrays(
    (
        pd.Series(letters.categories,dtype=letters).sample(20, replace=True),
        pd.Series(numbers.categories, dtype=numbers).sample(20, replace=True),
    ), 
    names=['nodes', 'edges']
).drop_duplicates()

In [111]:
multi


MultiIndex([('m', 9),
            ('b', 2),
            ('c', 9),
            ('f', 5),
            ('k', 4),
            ('o', 0),
            ('n', 9),
            ('a', 1),
            ('e', 7),
            ('h', 2),
            ('d', 1),
            ('o', 7),
            ('f', 6),
            ('p', 1),
            ('k', 5),
            ('j', 2),
            ('n', 6),
            ('a', 3),
            ('b', 9),
            ('k', 1)],
           names=['nodes', 'edges'])

In [112]:

multi.levels

FrozenList([['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [118]:
levi= pd.Series(np.nan, index=multi)#.sample(10)
# levi.loc[levi.sample(10).index]=0
levi.loc[levi.sample(15).index]=1
levi.index=multi
levi=levi.astype('Sparse')
levi

nodes  edges
m      9        1.0
b      2        1.0
c      9        1.0
f      5        1.0
k      4        1.0
o      0        1.0
n      9        1.0
a      1        1.0
e      7        NaN
h      2        NaN
d      1        1.0
o      7        1.0
f      6        NaN
p      1        NaN
k      5        1.0
j      2        1.0
n      6        1.0
a      3        NaN
b      9        1.0
k      1        1.0
dtype: Sparse[float64, nan]

In [119]:
levi

nodes  edges
m      9        1.0
b      2        1.0
c      9        1.0
f      5        1.0
k      4        1.0
o      0        1.0
n      9        1.0
a      1        1.0
e      7        NaN
h      2        NaN
d      1        1.0
o      7        1.0
f      6        NaN
p      1        NaN
k      5        1.0
j      2        1.0
n      6        1.0
a      3        NaN
b      9        1.0
k      1        1.0
dtype: Sparse[float64, nan]

In [120]:
I, p1, p2 = levi.sparse.to_coo(row_levels=['edges'], column_levels=['nodes'])
I

<9x13 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in COOrdinate format>

In [135]:
X=(I.T@I).toarray()
X=X-np.diag(np.diag(X))
X

array([[0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [136]:
np.sum(np.multiply(X[:,:,None],X[:,None]), axis=0)


array([[3., 2., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1.],
       [2., 4., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [2., 2., 3., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 3., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 2., 2., 0., 0., 0., 3., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1., 0., 0., 2., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]])

In [137]:
X.T@X

array([[3., 2., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1.],
       [2., 4., 2., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [2., 2., 3., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 3., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 2., 2., 0., 0., 0., 3., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1., 0., 0., 2., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]])

In [138]:
np.add(X.T[:,:,None],X[:,None])

array([[[0., 1., 1., ..., 0., 0., 0.],
        [1., 2., 2., ..., 1., 1., 1.],
        [1., 2., 2., ..., 1., 1., 1.],
        ...,
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.]],

       [[2., 1., 2., ..., 1., 1., 2.],
        [1., 0., 1., ..., 0., 0., 1.],
        [2., 1., 2., ..., 1., 1., 2.],
        ...,
        [1., 0., 1., ..., 0., 0., 1.],
        [1., 0., 1., ..., 0., 0., 1.],
        [2., 1., 2., ..., 1., 1., 2.]],

       [[2., 2., 1., ..., 1., 1., 1.],
        [2., 2., 1., ..., 1., 1., 1.],
        [1., 1., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [139]:
def min_sum(A,B):
    left=np.where(A>0, A, np.inf).copy()
    right=np.where(B>0, B, np.inf).copy()

    brdcst = np.add(left[:,:,None],right[:,None])
    return np.min(brdcst, axis=0)
# np.min(np.add(X[:,:,None],X[:,None]), axis=0)
min_sum(X.T,X)
# X.where()


array([[ 2.,  2.,  2., inf, inf, inf,  2., inf, inf, inf, inf, inf,  2.],
       [ 2.,  2.,  2., inf, inf, inf,  2., inf, inf, inf, inf, inf, inf],
       [ 2.,  2.,  2., inf, inf, inf,  2., inf, inf, inf, inf, inf,  2.],
       [inf, inf, inf,  2., inf, inf, inf,  2., inf, inf,  2., inf, inf],
       [inf, inf, inf, inf,  2., inf, inf,  2., inf, inf,  2., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [ 2.,  2.,  2., inf, inf, inf,  2., inf, inf, inf, inf, inf,  2.],
       [inf, inf, inf,  2.,  2., inf, inf,  2., inf, inf,  2., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf,  2.,  2., inf, inf,  2., inf, inf,  2., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [ 2., inf,  2., inf, inf, inf,  2., inf, inf, inf, inf, inf,  2.]])

In [140]:
def tropical_pow(A, n=2):
    p = min_sum(A.T,A)
    if n<=0:
        return p
    else: 
        return tropical_pow(p,n=n-1)

tropical_pow(X, 3)

array([[16., 16., 16., inf, inf, inf, 16., inf, inf, inf, inf, inf, 16.],
       [16., 16., 16., inf, inf, inf, 16., inf, inf, inf, inf, inf, 16.],
       [16., 16., 16., inf, inf, inf, 16., inf, inf, inf, inf, inf, 16.],
       [inf, inf, inf, 16., 16., inf, inf, 16., inf, inf, 16., inf, inf],
       [inf, inf, inf, 16., 16., inf, inf, 16., inf, inf, 16., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [16., 16., 16., inf, inf, inf, 16., inf, inf, inf, inf, inf, 16.],
       [inf, inf, inf, 16., 16., inf, inf, 16., inf, inf, 16., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [inf, inf, inf, 16., 16., inf, inf, 16., inf, inf, 16., inf, inf],
       [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       [16., 16., 16., inf, inf, inf, 16., inf, inf, inf, inf, inf, 16.]])